# Dynamic Limit Orders

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import yfinance as yf
from ib_insync import *
import nest_asyncio
import logging
import datetime
import schedule
import time
import threading

In [3]:
nest_asyncio.apply()

# Connect to TWS (or gateway)
ib = IB()

ib.connect('127.0.0.1', 7496, clientId=1111) #7497 - Paper Trading, 7496 - Live trading

<IB connected to 127.0.0.1:7496 clientId=1111>

In [5]:
#Define ES contract
#contract = Future('ES', '202506', 'CME')
#contract = Future('DA', '202505', 'CME')
contract = Future('KC', '202505', 'NYBOT')
#contract = Future('SB', '202505', 'NYBOT')
#contract = Future('LE', '202506', 'CME')
# Or for stocks:
#contract = Stock('TSLA', 'SMART', 'USD')

# Qualify the contract (ensures IB recognizes it)
ib.qualifyContracts(contract)

[Future(conId=565301279, symbol='KC', lastTradeDateOrContractMonth='20250519', multiplier='37500', exchange='NYBOT', currency='USD', localSymbol='KCK5', tradingClass='KC')]

In [7]:
quote = ib.reqMktData(contract, "", False, False)
depth = ib.reqMktDepth(contract, numRows=3)

In [9]:
first_bid = depth.domBids[0].price
second_bid = depth.domBids[1].price
bid_difference = round(first_bid - second_bid,4)
print(bid_difference)

0.05


## Market Depth Version:

In [41]:
min_tick_size = 0.025 # add instrument tick size

# Global variable to track position
position_size = 0

# Define a function to handle position updates
def update_position(account, contract, pos, avgCost):
    global position_size
    # Check if this is the contract we're trading (adjust comparison as needed)
    if contract.symbol == "KC":  # Replace with your contract's symbol
        position_size = pos
        print(f"Position updated: {position_size}")

# Assign the callback to the wrapper
ib.wrapper.position = update_position

my_bid = quote.bid + min_tick_size
order = LimitOrder("BUY", 1, my_bid)
ib.placeOrder(contract, order)

# Request position updates
ib.reqPositions()

# Start the IB event loop in a separate thread
threading.Thread(target=ib.run, daemon=True).start()

# Loop every 1 sec
while True:
    ib.sleep(0.35)
    
    if position_size == 0:  # Only proceed if no position
        # Get the second bid from the order book
        first_bid = depth.domBids[0].price
        second_bid = depth.domBids[1].price
        bid_difference = round(my_bid - second_bid, 2)
        print(f"Bid difference: {bid_difference}")

        # Case 1: Market best bid exceeds my bid, raise my bid
        if quote.bid > my_bid:
            new_bid = quote.bid + min_tick_size
            new_order = LimitOrder("BUY", 1, new_bid)
            ib.cancelOrder(order)
            ib.placeOrder(contract, new_order)
            print(f"Cancelled/placed order (raised): {new_bid}")
            my_bid = new_bid
            order = new_order

        # Case 2: Second bid is more than 1 tick below my bid, lower my bid
        elif bid_difference > min_tick_size:
            new_bid = second_bid + min_tick_size
            new_order = LimitOrder("BUY", 1, new_bid)
            ib.cancelOrder(order)
            ib.placeOrder(contract, new_order)
            print(f"Cancelled/placed order (lowered): {new_bid}")
            my_bid = new_bid
            order = new_order

        else:
            print("No adjustment needed.")
    else:
        print("Position held, skipping order placement.")
        break # Position = 1, stop looping

Bid difference: 0.03
Cancelled/placed order (lowered): 195.8
Bid difference: 0.03
Cancelled/placed order (lowered): 195.8
Bid difference: 0.03
Cancelled/placed order (lowered): 195.8
Bid difference: 0.03
Cancelled/placed order (lowered): 195.8
Bid difference: 0.03
Cancelled/placed order (lowered): 195.8
Bid difference: 0.03
Cancelled/placed order (lowered): 195.8
Bid difference: 0.03
Cancelled/placed order (lowered): 195.8
Position held, skipping order placement.


In [15]:
# Global variable to track position
position_size = 0

# Define a function to handle position updates
def update_position(account, contract, pos, avgCost):
    global position_size
    # Check if this is the contract we're trading (adjust comparison as needed)
    if contract.symbol == "ES":  # Replace with your contract's symbol
        position_size = pos
        print(f"Position updated: {position_size}")

# Assign the callback to the wrapper
ib.wrapper.position = update_position

my_bid = quote.bid - 0.75
order = LimitOrder("BUY", 1, my_bid)
ib.placeOrder(contract, order)

# Request position updates
ib.reqPositions()

# Start the IB event loop in a separate thread
threading.Thread(target=ib.run, daemon=True).start()

# Loop every 1 sec
while True:
    ib.sleep(0.35)

    if position_size == 0:  # Only proceed if no position
        # If best bid > my bid, cancel & update my bid
        if quote.bid > my_bid:
            new_bid = quote.bid - 0.75
            new_order = LimitOrder("BUY", 1, new_bid)
         
            # Cancel the old order and place the new order
            ib.cancelOrder(order)
            ib.placeOrder(contract, new_order)
            print("Cancelled order/placed new order:", new_bid)
            
            # Update the current bid and order for subsequent iterations 
            my_bid = new_bid
            order = new_order
        else:
            print("Quote.bid not > my_bid.")
    else:
        print("Position already held, skipping order placement.")

Position already held, skipping order placement.
Position already held, skipping order placement.
Position already held, skipping order placement.
Position already held, skipping order placement.
Position already held, skipping order placement.
Position already held, skipping order placement.
Position already held, skipping order placement.


KeyboardInterrupt: 